# Reporte Sprints



In [144]:
from IPython.display import HTML
from IPython.display import display        

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click aquí para ver el código"></form>''')




In [145]:
## CONFIGURACION
SPRINT_ACTUAL = "export9.csv"  
SPRINT_ANTERIOR = "export8.csv"  
#DATE_START = "20170515"  
#DATE_END = "20170526"  
#DATES_HOLIDAYS = []  
print("CONFIGURACIÓN")
print("SPRINT ACTUAL: "+SPRINT_ACTUAL)
print("SPRINT ANTERIOR: "+SPRINT_ANTERIOR)


CONFIGURACIÓN
SPRINT ACTUAL: export9.csv
SPRINT ANTERIOR: export8.csv


In [146]:
## IMPORT AND LOADING

import pandas as pd
import math

col_names = ["#","Estado","Proyecto","Tipo","% Realizado","Tiempo estimado","Tarjeta","Versión prevista"]
new_col_names = ["Issue","Estado","Proyecto","Tipo","Porcentaje","Tiempo","Tarjeta","Version"]
col_names2 = ["#","Estado","Proyecto","Tipo","% Realizado","Tiempo estimado","Versión prevista"]
new_col_names2 = ["Issue","Estado","Proyecto","Tipo","Porcentaje","Tiempo","Version"]

grupos_tarjetas = {"SVV":"SVV",
                   "TC":"TC",
                   "MON":"SVE",
                   "SVD":"SVE",
                   "PC":"PC"}


finished_states = ["Pte Validacion","Corregida","Resuelta"]
blocked_states = ["Bloqueada"]
special_types = ["Soporte","Unexpected"]


dat = pd.read_csv(SPRINT_ACTUAL,sep=";",header=0,usecols=col_names,encoding="iso-8859-15",decimal=",")
dat = dat.rename(columns=dict(zip(col_names,new_col_names)))
datPrev = pd.read_csv(SPRINT_ANTERIOR,sep=";",header=0,usecols=col_names2,encoding="iso-8859-15",decimal=",")
datPrev = datPrev.rename(columns=dict(zip(col_names2,new_col_names2)))

dat.loc[dat["Issue"] == 14169]


,Issue,Estado,Proyecto,Tipo,Version,Porcentaje,Tiempo,Tarjeta
2,14169,Resuelta,MetroArgel,Soporte,SprintSoporte_17_20,100,8.0,NaN


## Peticiones con errores 

#### Tiempo de la tarea

In [147]:
def check_bad_time(DF):
    if pd.isnull(DF["Tiempo"]).any() | (DF["Tiempo"] == 0).any():
        display("Lista de peticiones con tiempo inválido o 0 (eliminadas de la lista a partir de ahora)")
        index = DF.loc[(pd.isnull(dat["Tiempo"])) | (DF["Tiempo"] == 0),:].index
        display(DF.loc[index,:])
        DF.drop(index,inplace=True)
    else:
        display("Todas las peticiones tienen tiempos correctos")

print("Sprint Actual: "+SPRINT_ACTUAL)
check_bad_time(dat)
print("Sprint Anterior: "+SPRINT_ANTERIOR)
check_bad_time(datPrev)


Sprint Actual: export9.csv


'Lista de peticiones con tiempo inválido o 0 (eliminadas de la lista a partir de ahora)'

,Issue,Estado,Proyecto,Tipo,Version,Porcentaje,Tiempo,Tarjeta
37,14092,Pte Validacion,GreenLine,Tarea,SprintSoporte_17_20,100,NaN,NaN
43,14056,Pte Validacion,R168,Tarea,SprintSoporte_17_20,100,NaN,NaN
73,13909,Nueva,MetroArgel,Spike,SprintMARG_17_20,0,NaN,MON
125,13098,Nueva,GreenLine,Spike,ProductBackLogGL,0,NaN,NaN
132,12840,Nueva,MNR,User Story,ProductBackLogMNR,0,NaN,NaN
145,12177,Nueva,MNR,User Story,ProductBackLogMNR,0,NaN,NaN
146,11904,Nueva,MNR,User Story,ProductBackLogMNR,0,NaN,NaN


Sprint Anterior: export8.csv


'Todas las peticiones tienen tiempos correctos'

#### Inconsistencias Estado/Porcentaje

In [148]:
def check_inconsistent_state(DF):
    inconsistent = DF[(DF["Estado"].isin(finished_states)) & (DF["Porcentaje"] < 100)]
    if inconsistent.empty:
        print("Todas las peticiones tienen estado/porcentaje consistente")
    else:
        print("Lista de peticiones con inconsistencias en Estado/Porcentaje (se eliminan)")
        display(inconsistent)
        DF.drop(inconsistent.index,inplace=True)

print("Sprint Actual: "+SPRINT_ACTUAL)
check_inconsistent_state(dat)
print("Sprint Anterior: "+SPRINT_ANTERIOR)
check_inconsistent_state(datPrev)

Sprint Actual: export9.csv
Lista de peticiones con inconsistencias en Estado/Porcentaje (se eliminan)


,Issue,Estado,Proyecto,Tipo,Version,Porcentaje,Tiempo,Tarjeta
94,13800,Pte Validacion,MetroArgel,Spike,SprintMARG_17_20,90,16.0,SVD
131,12854,Pte Validacion,CIVIA - CCTV,User Story,ProductBackLogCIVIA,90,16.0,NaN


Sprint Anterior: export8.csv
Todas las peticiones tienen estado/porcentaje consistente


#### Inconsistencias Tipo Soporte / Sprint Soporte

In [149]:
def check_soporte(DF):
    soporte = DF[(DF["Tipo"] == "Soporte") & (~DF["Version"].str.contains("Soporte"))]
    if soporte.empty:
        print("Todas las tareas de soporte están en la sprint correcta")
    else:
        print("Lista de tareas de tipo soporte en sprint incorrecta (se mantienen)")
        display(soporte)

print("Sprint Actual: "+SPRINT_ACTUAL)
check_soporte(dat)
print("Sprint Anterior: "+SPRINT_ANTERIOR)
check_soporte(datPrev)

Sprint Actual: export9.csv
Todas las tareas de soporte están en la sprint correcta
Sprint Anterior: export8.csv
Todas las tareas de soporte están en la sprint correcta


#### Inconsistencias en Tarjeta

In [150]:
def check_tarjeta(DF):
    tarjetaNaN = DF.loc[pd.isnull(DF["Tarjeta"])]
    if tarjetaNaN.empty:
        print("Todas las peticiones tienen la tarjeta asignada")
    else:
        print("Lista de peticiones con la tarjeta no asignada (se mantienen)")
        display(tarjetaNaN)
        
print("Sprint Actual: "+SPRINT_ACTUAL)
check_tarjeta(dat)

Sprint Actual: export9.csv
Lista de peticiones con la tarjeta no asignada (se mantienen)


,Issue,Estado,Proyecto,Tipo,Version,Porcentaje,Tiempo,Tarjeta
2,14169,Resuelta,MetroArgel,Soporte,SprintSoporte_17_20,100,8.0,NaN
4,14154,En progreso,FGC254,User Story,ProductBackLogFGC254,0,1.5,NaN
5,14153,Pte Validacion,CCTV-SVD-MON-TC,Tarea,SprintSoporte_17_20,100,2.0,NaN
6,14150,Nueva,FGC112-213,Spike,ProductBackLogFGC112213,0,24.0,NaN
7,14149,Nueva,FGC112-213,Spike,ProductBackLogFGC112213,0,24.0,NaN
11,14140,En progreso,MNR,Soporte,SprintSoporte_17_20,100,24.0,NaN
12,14139,Nueva,PlantillaSVD-MON,User Story,PlantillaSVD-MONBacklog,0,8.0,NaN
39,14073,Resuelta,GreenLine,Soporte,SprintSoporte_17_20,100,1.0,NaN
40,14065,Resuelta,GreenLine,Soporte,SprintSoporte_17_20,100,2.0,NaN
42,14057,Resuelta,CINCINNATI CCTV,Soporte,SprintSoporte_17_20,100,6.0,NaN


## Estadísticas por proyecto

Se calculan las horas totales del proyecto, eliminando las tareas de tipo Soporte o Unexpected.

TiempoFinalizado es el tiempo calculado a partir del porcentaje.  
TiempoPendiente es la diferencia entre el tiempo real y el finalizado.

In [151]:
#dat.groupby("Proyecto").loc["Tiempo","TiempoReal"].sum().reset_index()
# for proy in dat.Proyecto.unique():
#     print("Proyecto {:>20s} ==> {:>6.1f} horas".format(proy,dat.loc[dat["Proyecto"] == proy,"Tiempo"].sum()))

dat["TiempoFinalizado"] = dat["Tiempo"]*dat["Porcentaje"]/100;
dat["TiempoPendiente"] =dat["Tiempo"]*(100-dat["Porcentaje"])/100
datPrev["TiempoFinalizado"] = datPrev["Tiempo"]*datPrev["Porcentaje"]/100;
datPrev["TiempoPendiente"] = datPrev["Tiempo"]*(100-datPrev["Porcentaje"])/100

# Se asigna la tarjeta correcta
dat["Tarjeta"] = dat["Tarjeta"].map(grupos_tarjetas).fillna("UNK").astype(str)
#display(dat)

display(dat.groupby(["Proyecto","Version","Tarjeta"])["Tiempo","TiempoFinalizado","TiempoPendiente"].sum())
        

Tiempo  TiempoFinalizado  \
Proyecto         Version                 Tarjeta                             
CCTV-SVD-MON-TC  SprintSoporte_17_20     UNK         2.0               2.0   
CINCINNATI CCTV  SprintSoporte_17_20     UNK         8.0               8.0   
CIVIA - CCTV     SprintCIVIA_17_20       UNK        64.0               3.2   
FGC112-213       ProductBackLogFGC112213 SVV        40.0               0.0   
                                         UNK        48.0               0.0   
                 SprintFGC_17_20         TC         24.0               0.0   
FGC254           ProductBackLogFGC254    SVV        56.0               4.0   
                                         TC         16.0               0.0   
                                         UNK         1.5               0.0   
                 SprintFGC254_17_20      SVV       220.0             212.0   
                                         TC         41.0              41.0   
GreenLine        ProductBackLogGL        UNK        40.0               0.0   
                 SprintGL_17_20          SVE        40.0              40.0   
                 SprintSoporte_17_20     UNK         3.0               3.0   
Guadalajara      SprintSoporte_17_20     UNK         5.5               5.5   
MNR              ProductBackLogMNR       UNK       336.0               0.0   
                 SprintSoporte_17_20     UNK        45.0              45.0   
MetroArgel       ProductBackLogMArgel    SVE        16.0               0.0   
                                         UNK       108.0               0.0   
                 SprintMARG_17_20        SVE        90.0              68.4   
                                         SVV        56.0              44.0   
                                         TC         16.0               0.0   
                                         UNK       104.0             100.0   
                 SprintSoporte_17_20     SVE         4.0               4.0   
                                         UNK        24.0              24.0   
MetroMadrid2000B SprintMM2008B_17_20     TC        112.0              60.0   
                                         UNK        24.0              24.0   
PRODUCTO         SprintPROD_17_20        SVE        40.0              32.0   
                                         SVV        16.0              16.0   
Panama           SprintPanama_17_20      SVE        56.0              56.0   
PlantillaSVD-MON PlantillaSVD-MONBacklog SVE       294.0               0.0   
                                         SVV        20.0               0.0   
                                         UNK         8.0               0.0   
Toluca           SprintSoporte_17_20     UNK         2.5               2.5   
                 SprintTOL_17_20         SVE        24.0              12.0   

                                                  TiempoPendiente  
Proyecto         Version                 Tarjeta                   
CCTV-SVD-MON-TC  SprintSoporte_17_20     UNK                  0.0  
CINCINNATI CCTV  SprintSoporte_17_20     UNK                  0.0  
CIVIA - CCTV     SprintCIVIA_17_20       UNK                 60.8  
FGC112-213       ProductBackLogFGC112213 SVV                 40.0  
                                         UNK                 48.0  
                 SprintFGC_17_20         TC                  24.0  
FGC254           ProductBackLogFGC254    SVV                 52.0  
                                         TC                  16.0  
                                         UNK                  1.5  
                 SprintFGC254_17_20      SVV                  8.0  
                                         TC                   0.0  
GreenLine        ProductBackLogGL        UNK                 40.0  
                 SprintGL_17_20          SVE                  0.0  
                 SprintSoporte_17_20     UNK                  0.0  
Guadalajara      SprintSoporte_17_20     UNK                  0.0  
MNR    

### Suma de horas pendientes por Proyecto

Se suman todos los tiempos pendientes de las distintas versiones (Backlog y Sprint). No se han quitado las tareas de soporte, pero estas deberían tener como TiempoPendiente 0 por lo que no afectarían al resultado.

In [152]:
display(dat.groupby(["Proyecto"])["Tiempo","TiempoPendiente"].sum())

,Tiempo,TiempoPendiente
Proyecto,,
CCTV-SVD-MON-TC,2.0,0.0
CINCINNATI CCTV,8.0,0.0
CIVIA - CCTV,64.0,60.8
FGC112-213,112.0,112.0
FGC254,334.5,77.5
GreenLine,83.0,40.0
Guadalajara,5.5,0.0
MNR,381.0,336.0
MetroArgel,418.0,177.6


### Suma de horas de Soporte

In [153]:
dat.loc[dat["Tipo"] == "Soporte",["Tiempo","TiempoFinalizado"]].sum()

Tiempo              88.0
TiempoFinalizado    88.0
dtype: float64

## Sprint Actual 

Los datos siguientes se refieren al sprint actual. Horas realizadas, teniendo en cuenta las horas realizadas hasta el sprint anterior. 

In [154]:
#Extraemos solo el proyecto y los tiempos de las tareas que no están en Backlog ni Soporte
dat["TiempoSprintAnterior"] = dat["Issue"].map(datPrev.set_index("Issue")["TiempoFinalizado"]).fillna(0).astype(int)
sprint=dat[~dat["Version"].str.contains('BackLog|Soporte|Backlog')]
sprintT=sprint.groupby("Proyecto")["Tiempo","TiempoFinalizado","TiempoSprintAnterior"].sum()
sprintT["TiempoFinalizadoReal"] = sprintT["TiempoFinalizado"]-sprintT["TiempoSprintAnterior"]
display(sprintT)
display("Tiempos Totales:")
display(sprintT.sum(numeric_only=True))



,Tiempo,TiempoFinalizado,TiempoSprintAnterior,TiempoFinalizadoReal
Proyecto,,,,
CIVIA - CCTV,64.0,3.2,3,0.2
FGC112-213,24.0,0.0,0,0.0
FGC254,261.0,253.0,0,253.0
GreenLine,40.0,40.0,0,40.0
MetroArgel,266.0,212.4,68,144.4
MetroMadrid2000B,136.0,84.0,0,84.0
PRODUCTO,56.0,48.0,0,48.0
Panama,56.0,56.0,0,56.0
Toluca,24.0,12.0,0,12.0


'Tiempos Totales:'

Tiempo                  927.0
TiempoFinalizado        708.6
TiempoSprintAnterior     71.0
TiempoFinalizadoReal    637.6
dtype: float64

### Resumen Sprint

In [155]:
print("Tiempo total realizado en este sprint:")
print("Tiempo tareas del Sprint: ")
tSprint = sprintT.sum(numeric_only=True)["TiempoFinalizadoReal"]
display(tSprint)
print("Tiempo tareas Soporte: ")
tSoporte = dat.loc[dat["Tipo"] == "Soporte",["Tiempo","TiempoFinalizado"]].sum()["TiempoFinalizado"]
display(tSoporte)
print("Tiempo Total: ")
display(tSprint+tSoporte)



Tiempo total realizado en este sprint:
Tiempo tareas del Sprint: 


637.60000000000002

Tiempo tareas Soporte: 


88.0

Tiempo Total: 


725.60000000000002